In [ ]:
import uproot
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import gaussian_kde


In [ ]:
# fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_1of10.root"
# #fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_1p35_1of10.root"
# _tree = uproot.open(fName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["jetTree"]

# df = _tree.pandas.df("jet*")
# dfGen = _tree.pandas.df(["genJetPt","genJetEta"])


In [ ]:
maxN = 0
for i in range(1,3):
    print(i,maxN)
    fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_%iof10.root"%i
    fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_1p35_%iof10.root"%i
#    fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_CTC48_stcMipPt_gt_-1p00_%iof10.root"%i
    #fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_STC16_%iof10.root"%i
#    fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_BestChoice_%iof10.root"%i
    _tree = uproot.open(fName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["jetTree"]
    this_df = _tree.pandas.df("jet*")
    this_dfGen = _tree.pandas.df(["genJetPt","genJetEta"])

    this_df.reset_index(inplace=True)
    this_dfGen.reset_index(inplace=True)
    this_df.entry += maxN
    this_dfGen.entry += maxN
    this_df.set_index(['entry','subentry'],inplace=True)
    this_dfGen.set_index(['entry','subentry'],inplace=True)
    
    if i==1:
        df = this_df.copy(deep=True)        
        dfGen = this_dfGen.copy(deep=True)
    else:
        df = pd.concat([df,this_df])
        dfGen = pd.concat([dfGen,this_dfGen])
    maxN = df.index.levels[0].max()+1


In [ ]:
df = pd.merge(df, dfGen, left_on=['entry','jetGenJetMatch'],right_on=['entry','subentry'], how='left')

df['VBF'] = df.jetGenJetMatch>-1
df['PU'] = (df.jetGenJetMatch==-1) & (df.jetMinGenJetDR>0.5)

df['jetPt_PUcorr'] = df.jetPt - (df.jetPtR05 + df.jetPtR06)/1.25

df['jetPt_PUcorr2'] = df.jetPt - (df.jetPtR03 + df.jetPtR04)/0.75

In [ ]:
# df_135 = df.copy(deep=True)

In [ ]:
# pd.merge(df_0,df_135, on=['entry',])

In [ ]:
for eta in np.arange(15,30)*.1:
    sel = df.VBF & (abs(df.jetEta)>eta) & (abs(df.jetEta)<(eta+.1))

    x = df.genJetPt[sel]
    y = df.jetPt[sel]#-df.genJetPt[sel]

    yi = y[x>0]
    xi = x[x>0]

    slope, intercept, r_value, p_value, std_err = stats.linregress(xi,yi)
#    print("%.1f to %.1f"%(,x+.1))

    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)


    print("%.1f to %.1f, slope=%.4f, intercept=%.2f"%(eta,eta+.1,slope,intercept))
    fig, ax = plt.subplots()
    ax.scatter(x, y, c=z, s=10, edgecolor='',cmap=plt.cm.jet)
    plt.plot(x, slope*x+intercept,'k')
    plt.axis([-50,800,-50,800])
#     plt.show()

#    plt.scatter(x,y,'o',markersize=1.5)

    plt.text(10,700,s="tc > 0; %.1f<|$\eta$|<%.1f\ny= %.4f x  + %.4f"%(eta,eta+.1,slope,intercept))
    plt.xlabel("Gen Jet Pt")
    plt.ylabel("Reco Jet Pt")
    plt.savefig("tc_0_eta_%i.pdf"%int(eta*10))

In [ ]:
sel = df.VBF & (abs(df.jetEta)>1.9) & (abs(df.jetEta)<2.6)

x = df.genJetPt[sel]
y = df.jetPt[sel]#-df.genJetPt[sel]

yi = y[x>0]
xi = x[x>0]

slope, intercept, r_value, p_value, std_err = stats.linregress(xi,yi)

print(slope,intercept)

plt.plot(x,y,'o',markersize=1.5)
plt.plot(x, slope*x+intercept,'k')

plt.text(0,y.max()*.9,s="tc > 0; 1.9<|$\eta$|<2.6\ny= %.4f x  + %.4f"%(slope,intercept))
plt.xlabel("Gen Jet Pt")
plt.ylabel("Reco Jet Pt")
#plt.savefig("tc_0mipT_middle.pdf")

In [ ]:
sel = df.VBF & ((abs(df.jetEta)<1.9) | (abs(df.jetEta)>2.6))

x = df.genJetPt[sel]
y = df.jetPt[sel]#-df.genJetPt[sel]

yi = y[x>0]
xi = x[x>0]

slope, intercept, r_value, p_value, std_err = stats.linregress(xi,yi)

print(slope,intercept)

plt.plot(x,y,'o',markersize=1.5)
plt.plot(x, slope*x+intercept,'k')
plt.text(0,y.max()*.9,s="tc > 0; 1.9>|$\eta$| or |$\eta$|>2.6\ny= %.4f x  + %.4f"%(slope,intercept))
plt.xlabel("Gen Jet Pt")
plt.ylabel("Reco Jet Pt")
#plt.savefig("tc_0mipT_edge.pdf")

In [ ]:
#sns.regplot(x=df.genJetPt[df.VBF],y=df.jetPt[df.VBF],x_bins=np.arange(0,700,10))

In [ ]:
sel = (abs(df.jetEta)>1.9) & (abs(df.jetEta)<2.6)

df['A'] = np.maximum(df.jetPtR01,0.00001)
df['B'] = df.jetPtR02 + df.jetPtR03 + df.jetPtR04
df['C'] = df.jetPtR05 + df.jetPtR06

#df['newIso'] = (df.B - 15./20*df.C)/(df.A - 1./20*df.C)
df['newIso'] = df.B/df.A
bins=np.arange(0,25,.5)
plt.hist(df.newIso[df.PU&sel],bins=bins,normed=True);
plt.hist(df.newIso[df.VBF&sel],bins=bins,normed=True);
#plt.hist(df.newIso[df.PU],bins=np.arange(0,10,.1),normed=True);
plt.figure()
x, y, area, sig_area = make_roc(df.newIso[df.VBF&sel],df.newIso[df.PU&sel],binning=bins)
print(area)
plt.plot(x,y)
plt.text(.5,.2,s='AUC=%.4f'%area)


In [ ]:
sel = (abs(df.jetEta)>1.9) & (abs(df.jetEta)<2.6)

df['A'] = df.jetPtR01
df['B'] = df.jetPtR02# + df.jetPtR03 + df.jetPtR04
df['C'] = df.jetPtR05 + df.jetPtR06

#df['newIso'] = (df.B - 15./20*df.C)/(df.A - 1./20*df.C)
df['newIso'] = df.B/df.A

bins = np.arange(0,7,.1)
plt.hist(df.newIso[df.PU&sel],bins=bins,normed=True);
plt.hist(df.newIso[df.VBF&sel],bins=bins,normed=True);
#plt.hist(df.newIso[df.PU],bins=np.arange(0,10,.1),normed=True);
x, y, area, sig_area = make_roc(df.newIso[df.VBF&sel],df.newIso[df.PU&sel],binning=bins)
print(area)
plt.figure()
plt.plot(x,y)

In [ ]:
sel = (abs(df.jetEta)>1.9) & (abs(df.jetEta)<2.6)

df['A'] = df.jetPtR01
df['B'] = df.jetPtR02 #+ df.jetPtR03
df['C'] = df.jetPtR05 + df.jetPtR06

#df['newIso'] = (df.B - 8./20*df.C)/(df.A - 1./20*df.C)
df['newIso'] = df.B/df.A

plt.hist(df.newIso[df.PU&sel],bins=np.arange(0,25,.5),normed=True);
plt.hist(df.newIso[df.VBF&sel],bins=np.arange(0,25,.5),normed=True);
#plt.hist(df.newIso[df.PU],bins=np.arange(0,10,.1),normed=True);

In [ ]:
from sklearn.metrics import auc

def calc_hratio(dataset1, dataset2, binning): # calculate likelihood ratio per bin (s/s+b)
    h1, _ = np.histogram(dataset1, bins=binning, density=True)
    h2, _ = np.histogram(dataset2, bins=binning, density=True)
    mask = (h1 > 0) & (h2 > 0)
    h1 = h1[mask]
    h2 = h2[mask]
    lratio = h1/(h1 + h2)
    return lratio, h1, h2

def make_roc(dataset1, dataset2, binning):
    lratio, h1, h2 = calc_hratio(dataset1, dataset2, binning)
    fig, ax = plt.subplots()
    hratio, xxx,xxxx = ax.hist([lratio, lratio], bins=20, # constructing likelihood discriminator
                                          alpha=0.5,
                                          color=['C0', 'C1'],
                                          density=True,
                                          stacked=True,
                                          label=['matched VBF, 0PU', 'matched g-Jets, 0PU'],
                                          weights=[h1, h2] # weight by sig/bg probability
                                         );
    ax.set_xlabel('likelihood discriminator')
    #ax.text(0.10,10.,r'Threshold = 0')
    ax.legend();
    #plt.close()
    l1 = hratio[0]
    l2 = hratio[1] - hratio[0]
    m = len(dataset1)
    n = len(dataset2)
    x = 1 - np.cumsum(l2)/l2.sum() # constructing ROC
    y = 1 - np.cumsum(l1)/l1.sum()
    x = np.insert(x, 0, 1) # ensuring ROC covers 0 and 1 for well-defined AUC
    y = np.insert(y, 0, 1)
    area = auc(x, y) # AUC from scikit learn
    # uncertainty calculation
    pxxy = area/(2.-area)
    pxyy = 2*(area**2)/(1.+area)
    numer = area*(1-area) + (m-1)*(pxxy-area**2) + (n-1)*(pxyy-area**2)
    denom = m*n
    sig_area = np.sqrt(numer/denom)
    return x, y, area, sig_area
 

In [ ]:
sel = (abs(df.jetEta)>1.9) & (abs(df.jetEta)<2.6)

df['A'] = np.maximum(df.jetPtR01,0.00001)
df['B'] = df.jetPtR02 + df.jetPtR03 + df.jetPtR04
df['C'] = df.jetPtR05 + df.jetPtR06

#df['newIso'] = (df.B - 15./20*df.C)/(df.A - 1./20*df.C)
df['newIso'] = df.B/df.A
bins=np.arange(0,25,.1)
plt.hist(df.newIso[df.PU&sel],bins=bins,normed=True);
plt.hist(df.newIso[df.VBF&sel],bins=bins,normed=True);
#plt.hist(df.newIso[df.PU],bins=np.arange(0,10,.1),normed=True);
plt.figure()
_x = []
_y = []
_auc = []
for eta in np.arange(15,30)*0.1:
    sel = (abs(df.jetEta)>=eta) & (abs(df.jetEta)<(eta+0.1))
    gensel = (abs(dfGen.genJetEta)>=eta) & (abs(dfGen.genJetEta)<(eta+0.1))
    x, y, area, sig_area = make_roc(df.newIso[df.VBF&sel],df.newIso[df.PU&sel],binning=bins)
    print("$\eta$<%.1f, %i of %i, %.3f"%(eta, sum(df.VBF&sel),sum(gensel),sum(df.VBF&sel)/sum(gensel)))
    #    print(area)
    _x.append(x)
    _y.append(y)
    _auc.append(area)
#    plt.plot(x,y)
#plt.text(.5,.2,s='AUC=%.4f'%area)
fig, ax = plt.subplots()
etas = np.round(np.arange(15,30)*0.1,1)
for i,eta in enumerate(np.arange(15,30)*0.1):
    print(_auc[i])
    plt.plot(_x[i],_y[i],label='$\eta$<%.1f'%eta)
    plt.text(.7,.7-i*0.05,s='$\eta$<%.1f: AUC=%.4f'%(eta+.1,_auc[i]))

#plt.legend(etas)
#ax.legend(loc='right', bbox_to_anchor=(1, 0.5))
ax.legend(loc='upper left', bbox_to_anchor= (1.01, 1.01), ncol=2, 
            borderaxespad=0, frameon=False)


In [ ]:

fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_%iof10.root"%i
fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_1p35_%iof10.root"%i
fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_CTC48_stcMipPt_gt_-1p00_%iof10.root"%i
fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_STC16_%iof10.root"%i


In [ ]:
algos=['Threshold 0', 'Threshold 1.35', 'CTC48', 'STC4-STC16','Best Choice']
saveName=['tc_0', 'tc_135', 'CTC48', 'STC16','BestChoice']
for algo in range(4,5):
    algoName = algos[algo]
    maxN = 0
    for i in range(1,11):
        print(i,maxN)
        if algo == 0:
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_%iof10.root"%i
        elif algo==1:
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_1p35_%iof10.root"%i
        elif algo==2:
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_CTC48_stcMipPt_gt_-1p00_%iof10.root"%i
        elif algo==3:
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_STC16_%iof10.root"%i
        elif algo==4:
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_BestChoice_%iof10.root"%i
        _tree = uproot.open(fName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["jetTree"]

        this_df = _tree.pandas.df("jet*")
        this_dfGen = _tree.pandas.df(["genJetPt","genJetEta"])

        this_df.reset_index(inplace=True)
        this_dfGen.reset_index(inplace=True)
        this_df.entry += maxN
        this_dfGen.entry += maxN
        this_df.set_index(['entry','subentry'],inplace=True)
        this_dfGen.set_index(['entry','subentry'],inplace=True)

        if i==1:
            df = this_df.copy()        
            dfGen = this_dfGen.copy()
        else:
            df = pd.concat([df,this_df])
            dfGen = pd.concat([dfGen,this_dfGen])
        maxN = df.index.levels[0].max()+1


    df = pd.merge(df, dfGen, left_on=['entry','jetGenJetMatch'],right_on=['entry','subentry'], how='outer')

    df['VBF'] = df.jetGenJetMatch>-1
    df['PU'] = (df.jetGenJetMatch==-1) & (df.jetMinGenJetDR>0.5)

    df['jetPt_PUcorr'] = df.jetPt - (df.jetPtR05 + df.jetPtR06)/1.25

    df['jetPt_PUcorr2'] = df.jetPt - (df.jetPtR03 + df.jetPtR04)/0.75
    
    
    for eta in np.arange(15,30)*.1:
        sel = df.VBF & (abs(df.jetEta)>eta) & (abs(df.jetEta)<(eta+.1))

        x = df.genJetPt[sel]
        y = df.jetPt[sel]#-df.genJetPt[sel]

        yi = y[x>0]
        xi = x[x>0]

        slope, intercept, r_value, p_value, std_err = stats.linregress(xi,yi)
    #    print("%.1f to %.1f"%(,x+.1))

        xy = np.vstack([x,y])
        z = gaussian_kde(xy)(xy)


        print("%.1f to %.1f, slope=%.4f, intercept=%.2f"%(eta,eta+.1,slope,intercept))
        fig, ax = plt.subplots()
        ax.scatter(x, y, c=z, s=10, edgecolor='',cmap=plt.cm.jet)
        plt.plot(x, slope*x+intercept,'k')
        plt.axis([-50,800,-50,800])
    #     plt.show()

    #    plt.scatter(x,y,'o',markersize=1.5)

        plt.text(10,700,s="%s; %.1f<|$\eta$|<%.1f\ny= %.4f x  + %.4f"%(algoName,eta,eta+.1,slope,intercept))
        plt.xlabel("Gen Jet Pt")
        plt.ylabel("Reco Jet Pt")
        plt.savefig("%s_eta_%i.png"%(saveName[algo],int(eta*10)))
    

In [ ]:
algos=['Threshold 0', 'Threshold 1.35', 'STC4-STC16', 'CTC48','BestChoice']
outputName = "EfficienciesVsEta.pdf"
algos=['Threshold 0', 'Threshold 0.45','Threshold 0.90','Threshold 1.35', 'Threshold 2.50', 'Threshold 5.00', 'Threshold 7.50', 'Threshold 10.00', 'Threshold 15.00']
outputName = "EfficienciesVsEta_Thresholds.pdf"

algos=['Threshold 1.35 PU 0', 'Threshold 1.35 PU 50', 'Threshold 1.35 PU 100', 'Threshold 1.35 PU 140', 'Threshold 1.35 PU 180', 'Threshold 1.35 PU 200']
#algos=['Threshold 1.35 PU 50', 'Threshold 1.35 PU 100', 'Threshold 1.35 PU 140', 'Threshold 1.35 PU 200']
outputName = "EfficienciesVsEta_Threshold_135_PUScan.pdf"

# algos=['STC4-STC16','STC4-STC16_cut065','STC4-STC16_cut135','STC4-STC16_cut500','STC4-STC16_cut1000']
# outputName = "EfficienciesVsEta_STC16.pdf"
# algos=['CTC48','CTC48 >1.35','CTC48 >5.00','CTC48 >10.00','CTC48 >20.00']
# outputName = "EfficienciesVsEta_CTC48.pdf"

etas = np.arange(15,30)*.1
eff = {}
error = {}
plt.figure()
for algo in algos:
    df = None
    maxN = 0
    for i in range(1,11):
        print(i,maxN)
        if algo == 'Threshold 0':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_%iof10.root"%i
        elif algo=='Threshold 0.45':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_0p45_%iof10.root"%i
        elif algo=='Threshold 0.90':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_0p90_%iof10.root"%i
        elif algo=='Threshold 1.35' or algo=='Threshold 1.35 PU 200':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_1p35_%iof10.root"%i
        elif algo=='Threshold 2.50':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_2p50_%iof10.root"%i
        elif algo=='Threshold 5.00':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_5p00_%iof10.root"%i
        elif algo=='Threshold 7.50':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_7p50_%iof10.root"%i
        elif algo=='Threshold 10.00':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_10p00_%iof10.root"%i
        elif algo=='Threshold 15.00':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_tcMipPt_gt_15p00_%iof10.root"%i
        elif algo=='CTC48':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_CTC48_stcMipPt_gt_-1p00_%iof10.root"%i
        elif algo=='CTC48 >1.35':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_CTC48_stcMipPt_gt_1p35_%iof10.root"%i
        elif algo=='CTC48 >5.00':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_CTC48_stcMipPt_gt_5p00_%iof10.root"%i
        elif algo=='CTC48 >10.00':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_CTC48_stcMipPt_gt_10p00_%iof10.root"%i
        elif algo=='CTC48 >20.00':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_CTC48_stcMipPt_gt_20p00_%iof10.root"%i
        elif algo=='STC4-STC16':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_STC16_%iof10.root"%i
        elif algo=='STC4-STC16_cut065':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_STC16_stcMipPt_gt_0p65_%iof10.root"%i
        elif algo=='STC4-STC16_cut135':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_STC16_stcMipPt_gt_1p35_%iof10.root"%i
        elif algo=='STC4-STC16_cut500':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_STC16_stcMipPt_gt_5p00_%iof10.root"%i
        elif algo=='STC4-STC16_cut1000':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_superTC_STC16_stcMipPt_gt_10p00_%iof10.root"%i
        elif algo=='BestChoice':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_200PU_BestChoice_%iof10.root"%i
        elif algo=='Threshold 1.35 PU 0':
            if i>1: continue
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_0PU_tcMipPt_gt_1p35.root"
        elif algo=='Threshold 1.35 PU 50':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_50PU_tcMipPt_gt_1p35_%iof10.root"%i
        elif algo=='Threshold 1.35 PU 100':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_100PU_tcMipPt_gt_1p35_%iof10.root"%i
        elif algo=='Threshold 1.35 PU 140':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_140PU_tcMipPt_gt_1p35_%iof10.root"%i
        elif algo=='Threshold 1.35 PU 180':
            fName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/IsolationStudies/v3/fastJetClustering_VBF_180PU_tcMipPt_gt_1p35_%iof10.root"%i

        try:
            _tree = uproot.open(fName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["jetTree"]
        except:
            continue
        this_df = _tree.pandas.df("jet*")
        this_dfGen = _tree.pandas.df(["genJetPt","genJetEta"])

        this_df.reset_index(inplace=True)
        this_dfGen.reset_index(inplace=True)
        this_df.entry += maxN
        this_dfGen.entry += maxN
        this_df.set_index(['entry','subentry'],inplace=True)
        this_dfGen.set_index(['entry','subentry'],inplace=True)

        if not df is None:
            df = pd.concat([df,this_df])
            dfGen = pd.concat([dfGen,this_dfGen])
        else:
            df = this_df.copy()        
            dfGen = this_dfGen.copy()
        maxN = df.index.levels[0].max()+1


    df = pd.merge(df, dfGen, left_on=['entry','jetGenJetMatch'],right_on=['entry','subentry'], how='outer')

    df['VBF'] = df.jetMinGenJetDR<0.1
    df['PU'] = (df.jetGenJetMatch==-1) & (df.jetMinGenJetDR>0.5)

    df['jetPt_PUcorr'] = df.jetPt - (df.jetPtR05 + df.jetPtR06)/1.25

    df['jetPt_PUcorr2'] = df.jetPt - (df.jetPtR03 + df.jetPtR04)/0.75

    eff[algo] = []
    error[algo] = []
    for eta in etas:

        sel = df.VBF & (abs(df.genJetEta)>eta) & (abs(df.genJetEta)<(eta+.1))
        selGen = (abs(dfGen.genJetEta)>eta) & (abs(dfGen.genJetEta)<(eta+.1))
        x = sum(sel)/sum(selGen)
        eff[algo].append(x)
        error[algo].append((x*(1-x)/sum(selGen))**.5)
        
    eff[algo] = np.array(eff[algo])
    plt.errorbar(x=etas+.05,y=eff[algo],yerr=error[algo],xerr=.05,fmt='o',label=algo)
    print(sum(df.VBF)/dfGen.genJetPt.count())
plt.axis([1.5,3,-.05,1.3])

plt.legend(ncol=2)
plt.savefig(outputName)


In [ ]:
for eta in etas:
    selGen = (abs(dfGen.genJetEta)>eta) & (abs(dfGen.genJetEta)<(eta+.1))
    print(sum(selGen))

In [ ]:
df.head()

In [ ]:
import numba

@numba.jit
def jetCorr (x,y):
    if y==0:
        return 7*x
    elif y==1:
        return 2*x
    elif y==2:
        return 3*x
    else:
        return 4*x

df['jetEtaBin'] = np.digitize(abs(df.jetEta),np.array([1.5,2.0,2.5,3.0]))

df['jetCorr'] = jetCorr(df.jetPt,df.jetEtaBin)



In [ ]:
df.head()

In [ ]:
#df = pd.merge(df, dfGen, left_on=['entry','jetGenJetMatch'],right_on=['entry','subentry'], how='left')
etaMin = 1.5
etaMax = 3.0
print(sum(df.jetMinGenJetDR<0.1))
print(sum(df.jetMinGenJetDR<0.2))
print(sum(df.jetMinGenJetDR<0.3))
print(sum(df.jetMinGenJetDR<0.4))
print(sum((abs(dfGen.genJetEta)>etaMin) & (abs(dfGen.genJetEta)<etaMax)))
#df['PU'] = (df.jetGenJetMatch==-1) & (df.jetMinGenJetDR>0.5)
#sel = df.VBF & (abs(df.jetEta)>eta) & (abs(df.jetEta)<(eta+.1))
print()

In [ ]:
etaMin = 1.9
etaMax = 2.6

sel = (abs(df.jetEta)>etaMin) & (abs(df.jetEta)<etaMax)
a = plt.hist(df.jetMinGenJetDR[sel],bins=np.arange(0,2,.01))
plt.plot([.1,.1],[0,a[0].max()],c='black')
plt.plot([.2,.2],[0,a[0].max()],c='black')
plt.plot([.3,.3],[0,a[0].max()],c='black')
plt.plot([.4,.4],[0,a[0].max()],c='black')

tGen =  sum((abs(dfGen.genJetEta)>etaMin) & (abs(dfGen.genJetEta)<etaMax))

print(sum(a[0][:10])/tGen,sum(a[0][:20])/tGen,sum(a[0][:30])/tGen,sum(a[0][:40])/tGen)

In [ ]:
etaMin = 1.5
etaMax = 3.0

sel = (abs(df.jetEta)>etaMin) & (abs(df.jetEta)<etaMax)
a = plt.hist(df.jetMinGenJetDR[sel],bins=np.arange(0,2,.01))
plt.plot([.1,.1],[0,a[0].max()],c='black')
plt.plot([.2,.2],[0,a[0].max()],c='black')
plt.plot([.3,.3],[0,a[0].max()],c='black')
plt.plot([.4,.4],[0,a[0].max()],c='black')

tGen =  sum((abs(dfGen.genJetEta)>etaMin) & (abs(dfGen.genJetEta)<etaMax))

print(sum(a[0][:10])/tGen,sum(a[0][:20])/tGen,sum(a[0][:30])/tGen,sum(a[0][:40])/tGen)

In [ ]:
etaMin = 1.9
etaMax = 3.0

sel = (abs(df.jetEta)>etaMin) & (abs(df.jetEta)<etaMax)
a = plt.hist(df.jetMinGenJetDR[sel],bins=np.arange(0,2,.01))
plt.plot([.1,.1],[0,a[0].max()],c='black')
plt.plot([.2,.2],[0,a[0].max()],c='black')
plt.plot([.3,.3],[0,a[0].max()],c='black')
plt.plot([.4,.4],[0,a[0].max()],c='black')

tGen =  sum((abs(dfGen.genJetEta)>etaMin) & (abs(dfGen.genJetEta)<etaMax))

print(sum(a[0][:10])/tGen,sum(a[0][:20])/tGen,sum(a[0][:30])/tGen,sum(a[0][:40])/tGen)